# Load Packages & Data

In [1]:
import pkg_resources
try:
    pkg_resources.get_distribution('pingouin')
except pkg_resources.DistributionNotFound:
    !pip install pingouin

In [2]:
import pandas as pd
import numpy as np
import pingouin as pg
import scipy.stats as stats
import statsmodels.stats.multicomp as multi
import statsmodels.api as sm
pd.set_option('display.max.columns', None) # display all columns
pd.set_option('display.float_format', '{:.3f}'.format)

In [3]:
df = pd.read_excel('/content/drive/MyDrive/PT Research/data/final_updated_validation_dataset_v2.1.xlsx')
df.head()

,RECORD_ID,rater_1_id,rater_2_id,rater_1_R_HKAA,rater_2_R_HKAA,R_HKAA_diff_between_raters,R_HKAA_avg_of_raters,rater_1_L_HKAA,rater_2_L_HKAA,L_HKAA_diff_between_raters,L_HKAA_avg_of_raters,R_redcap_class,R_class_w_2_degree_cutoff,R_class_w_3_degree_cutoff,L_redcap_class,L_class_w_2_degree_cutoff,L_class_w_3_degree_cutoff,DL_R_FEMORAL_HEAD_COORDS,DL_R_KNEE_COORDS,DL_R_ANKLE_COORDS,DL_L_FEMORAL_HEAD_COORDS,DL_L_KNEE_COORDS,DL_L_ANKLE_COORDS,DL_R_HKAA,DL_L_HKAA,DL_R_CLASS,DL_L_CLASS,BMI,BMI_CLASS,LATERALITY,PROCEDURE_GROUP,PROCEDURE_LEVEL
0,19,4,6,179.025,178.740,0.285,178.882,177.659,177.354,0.305,177.506,1,0,0,1,1,0,"(793, 2463)","(717, 5009)","(692, 7202)","(1917, 2463)","(1824, 5021)","(1820, 7210)",178.943,178.023,0.000,0.000,36.620,3.000,1,1,0
1,30,2,6,164.255,164.602,-0.347,164.429,179.862,179.739,0.123,179.800,2,2,2,2,0,0,"(689, 2243)","(1016, 4879)","(678, 7206)","(1706, 2163)","(1695, 4826)","(1715, 7187)",164.664,179.278,2.000,0.000,27.120,1.000,0,1,0
2,44,1,6,171.452,170.470,0.982,170.961,171.135,170.486,0.649,170.810,1,1,1,1,1,1,"(653, 2557)","(671, 5113)","(1023, 7367)","(1765, 2519)","(1797, 5102)","(1448, 7356)",171.527,170.489,1.000,1.000,25.780,1.000,1,1,0
3,111,4,6,179.944,179.641,0.303,179.792,176.618,177.214,-0.596,176.916,1,1,0,1,1,1,"(658, 1877)","(633, 4750)","(611, 7278)","(1778, 1879)","(1825, 4755)","(1730, 7281)",180.000,176.910,0.000,1.000,NaN,NaN,1,1,0
4,128,1,6,175.928,177.414,-1.486,176.671,170.109,169.916,0.193,170.012,1,0,0,1,1,1,"(801, 2524)","(879, 5092)","(778, 7388)","(1859, 2564)","(1714, 5095)","(1951, 7357)",175.741,170.740,2.000,2.000,24.270,0.000,1,1,0


# Task 1: DL model check (190 images difficult set)

In [4]:
# check for which records DL has null values
df[df["DL_R_HKAA"].isnull() | df["DL_L_HKAA"].isnull()][["RECORD_ID", "DL_R_HKAA", "DL_L_HKAA"]]

,RECORD_ID,DL_R_HKAA,DL_L_HKAA
87,2084,NaN,177.134
128,239,NaN,NaN
139,565,NaN,NaN
140,600,NaN,162.339
158,1449,NaN,165.672
180,2316,NaN,171.573


## Count the number of bilateral (both leg not null) HKAAs found by DL

In [5]:
DL_bilateral_count = df[['DL_R_HKAA', 'DL_L_HKAA']].notnull().all(axis=1).sum()
print(DL_bilateral_count)

184


## Count the number of surgical HKAAs found by DL

In [6]:
# LATERALITY 0 = right, 1 = left, 2 = bilateral
# exclude records which DL has null values
df_surg_right = df[((df['LATERALITY'] == 0) | (df['LATERALITY'] == 2)) & df['DL_R_HKAA'].notnull()]
df_surg_left = df[((df['LATERALITY'] == 1) | (df['LATERALITY'] == 2)) & df['DL_L_HKAA'].notnull()]

In [7]:
# rater avg surgical leg dataframe
df_avg_surg_left = df_surg_left[['RECORD_ID', 'L_HKAA_avg_of_raters']].rename(columns = {'L_HKAA_avg_of_raters' : 'HKAA_surg'})
df_avg_surg_right = df_surg_right[['RECORD_ID', 'R_HKAA_avg_of_raters']].rename(columns = {'R_HKAA_avg_of_raters' : 'HKAA_surg'})
df_avg_surg = pd.concat([df_avg_surg_left, df_avg_surg_right])
df_avg_surg['rater_id'] = 1

In [8]:
# DL surgical leg dataframe
df_surg_left_dl = df_surg_left[['RECORD_ID', 'DL_L_HKAA']].rename(columns = {'DL_L_HKAA' : 'HKAA_surg'})
df_surg_right_dl = df_surg_right[['RECORD_ID', 'DL_R_HKAA']].rename(columns = {'DL_R_HKAA' : 'HKAA_surg'})
df_surg_dl = pd.concat([df_surg_left_dl, df_surg_right_dl])
df_surg_dl['rater_id'] = 2

In [9]:
# rater avg non-surgical leg dataframe
df_avg_nsurg_left = df_surg_left[df_surg_left['DL_R_HKAA'].notnull()][['RECORD_ID', 'R_HKAA_avg_of_raters']].rename(columns = {'R_HKAA_avg_of_raters' : 'HKAA_nsurg'})
df_avg_nsurg_right = df_surg_right[df_surg_right['DL_L_HKAA'].notnull()][['RECORD_ID', 'L_HKAA_avg_of_raters']].rename(columns = {'L_HKAA_avg_of_raters' : 'HKAA_nsurg'})
df_avg_nsurg = pd.concat([df_avg_nsurg_left, df_avg_nsurg_right])
df_avg_nsurg['rater_id'] = 1

In [10]:
# DL non-surgical leg dataframe
df_nsurg_left_dl = df_surg_left[df_surg_left['DL_R_HKAA'].notnull()][['RECORD_ID', 'DL_R_HKAA']].rename(columns = {'DL_R_HKAA' : 'HKAA_nsurg'})
df_nsurg_right_dl = df_surg_right[df_surg_right['DL_L_HKAA'].notnull()][['RECORD_ID', 'DL_L_HKAA']].rename(columns = {'DL_L_HKAA' : 'HKAA_nsurg'})
df_nsurg_dl = pd.concat([df_nsurg_left_dl, df_nsurg_right_dl])
df_nsurg_dl['rater_id'] = 2

In [11]:
DL_surg_count = len(df_surg_dl)
print(DL_surg_count)

191


## IRR b/w DL and average human rater on the surgical vs non-surgical leg

### Surgical Leg

In [12]:
# human rater
# calculate avg
df_avg_surg['HKAA_surg'].mean().round(3)

172.38

In [13]:
# dl
# calculate avg
df_surg_dl['HKAA_surg'].mean().round(3)

172.887

In [14]:
# combine rater and dl
df_surg = pd.concat([df_avg_surg, df_surg_dl])
df_surg

,RECORD_ID,HKAA_surg,rater_id
0,19,177.506,1
2,44,170.810,1
3,111,176.916,1
4,128,170.012,1
5,177,176.094,1
...,...,...,...
177,2070,178.100,2
179,2306,175.626,2
181,2359,173.734,2
183,2481,172.922,2


In [15]:
# IRR calculation
icc_surg = pg.intraclass_corr(data = df_surg, targets = 'RECORD_ID', raters = 'rater_id', ratings = 'HKAA_surg').round(3)
icc_surg.set_index('Type')

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.944,34.767,187,188,0.000,"[0.93, 0.96]"
ICC2,Single random raters,0.944,38.796,187,187,0.000,"[0.92, 0.96]"
ICC3,Single fixed raters,0.950,38.796,187,187,0.000,"[0.93, 0.96]"
ICC1k,Average raters absolute,0.971,34.767,187,188,0.000,"[0.96, 0.98]"
ICC2k,Average random raters,0.971,38.796,187,187,0.000,"[0.96, 0.98]"
ICC3k,Average fixed raters,0.974,38.796,187,187,0.000,"[0.97, 0.98]"


The interrater reliability is 0.974 for the surgical leg.

### Non-surgical Leg

In [16]:
# human rater
# calculate avg
df_avg_nsurg['HKAA_nsurg'].mean().round(3)

175.392

In [17]:
# dl
# calculate avg
df_nsurg_dl['HKAA_nsurg'].mean().round(3)

175.717

In [18]:
# combine rater and dl
df_nsurg = pd.concat([df_avg_nsurg, df_nsurg_dl])
df_nsurg

,RECORD_ID,HKAA_nsurg,rater_id
0,19,178.882,1
2,44,170.961,1
3,111,179.792,1
4,128,176.671,1
5,177,174.843,1
...,...,...,...
177,2070,169.121,2
179,2306,177.547,2
181,2359,177.052,2
183,2481,176.518,2


In [19]:
# IRR calculation
icc_nsurg = pg.intraclass_corr(data = df_nsurg, targets = 'RECORD_ID', raters = 'rater_id', ratings = 'HKAA_nsurg').round(3)
icc_nsurg.set_index('Type')

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.970,65.601,183,184,0.000,"[0.96, 0.98]"
ICC2,Single random raters,0.970,75.864,183,183,0.000,"[0.95, 0.98]"
ICC3,Single fixed raters,0.974,75.864,183,183,0.000,"[0.97, 0.98]"
ICC1k,Average raters absolute,0.985,65.601,183,184,0.000,"[0.98, 0.99]"
ICC2k,Average random raters,0.985,75.864,183,183,0.000,"[0.97, 0.99]"
ICC3k,Average fixed raters,0.987,75.864,183,183,0.000,"[0.98, 0.99]"


The interrater reliability is 0.972 for the surgical leg.

# Task 2: DL model check (122 images test set)

In [20]:
df2 = pd.read_csv('/content/drive/MyDrive/PT Research/data/updated_test_set_122_patients_DL_and_humans.csv')
df2.head()

,RECORD_ID,rater_1_id,rater_2_id,rater_1_R_HKAA,rater_2_R_HKAA,R_HKAA_diff_between_raters,R_HKAA_avg_of_raters,rater_1_L_HKAA,rater_2_L_HKAA,L_HKAA_diff_between_raters,L_HKAA_avg_of_raters,R_redcap_class,R_class_w_2_degree_cutoff,R_class_w_3_degree_cutoff,L_redcap_class,L_class_w_2_degree_cutoff,L_class_w_3_degree_cutoff,DL_R_FEMORAL_HEAD_COORDS,DL_R_KNEE_COORDS,DL_R_ANKLE_COORDS,DL_L_FEMORAL_HEAD_COORDS,DL_L_KNEE_COORDS,DL_L_ANKLE_COORDS,DL_R_HKAA,DL_L_HKAA,DL_R_CLASS,DL_L_CLASS,BMI,BMI_CLASS,LATERALITY,PROCEDURE_GROUP,PROCEDURE_LEVEL
0,19,4,6,179.025,178.740,0.285,178.882,177.659,177.354,0.305,177.506,1,0,0,1,1,0,"(793, 2463)","(717, 5009)","(692, 7202)","(1917, 2463)","(1824, 5021)","(1820, 7210)",178.943,178.023,0.000,0.000,36.620,3.000,1,1,0
1,30,2,6,164.255,164.602,-0.347,164.429,179.862,179.739,0.123,179.800,2,2,2,2,0,0,"(689, 2243)","(1016, 4879)","(678, 7206)","(1706, 2163)","(1695, 4826)","(1715, 7187)",164.664,179.278,2.000,0.000,27.120,1.000,0,1,0
2,44,1,6,171.452,170.470,0.982,170.961,171.135,170.486,0.649,170.810,1,1,1,1,1,1,"(653, 2557)","(671, 5113)","(1023, 7367)","(1765, 2519)","(1797, 5102)","(1448, 7356)",171.527,170.489,1.000,1.000,25.780,1.000,1,1,0
3,111,4,6,179.944,179.641,0.303,179.792,176.618,177.214,-0.596,176.916,1,1,0,1,1,1,"(658, 1877)","(633, 4750)","(611, 7278)","(1778, 1879)","(1825, 4755)","(1730, 7281)",180.000,176.910,0.000,1.000,NaN,NaN,1,1,0
4,128,1,6,175.928,177.414,-1.486,176.671,170.109,169.916,0.193,170.012,1,0,0,1,1,1,"(801, 2524)","(879, 5092)","(778, 7388)","(1859, 2564)","(1714, 5095)","(1951, 7357)",175.741,170.740,2.000,2.000,24.270,0.000,1,1,0


In [21]:
# check for which records DL has null values
df2[df2["DL_R_HKAA"].isnull() | df["DL_L_HKAA"].isnull()][["RECORD_ID", "DL_R_HKAA", "DL_L_HKAA"]]

<ipython-input-21-deccf4205eb6>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2[df2["DL_R_HKAA"].isnull() | df["DL_L_HKAA"].isnull()][["RECORD_ID", "DL_R_HKAA", "DL_L_HKAA"]]


,RECORD_ID,DL_R_HKAA,DL_L_HKAA
87,2084,NaN,177.134


## Count the number of bilateral (both leg not null) HKAAs found by DL

In [22]:
DL_bilateral_count = df2[['DL_R_HKAA', 'DL_L_HKAA']].notnull().all(axis=1).sum()
print(DL_bilateral_count)

121


## Count the number of surgical HKAAs found by DL

In [23]:
# LATERALITY 0 = right, 1 = left, 2 = bilateral
# exclude records which DL has null values
df_surg_right2 = df2[((df['LATERALITY'] == 0) | (df2['LATERALITY'] == 2)) & df2['DL_R_HKAA'].notnull()]
df_surg_left2 = df2[((df['LATERALITY'] == 1) | (df2['LATERALITY'] == 2)) & df2['DL_L_HKAA'].notnull()]

<ipython-input-23-3dc94fe6a557>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_surg_right2 = df2[((df['LATERALITY'] == 0) | (df2['LATERALITY'] == 2)) & df2['DL_R_HKAA'].notnull()]
<ipython-input-23-3dc94fe6a557>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_surg_left2 = df2[((df['LATERALITY'] == 1) | (df2['LATERALITY'] == 2)) & df2['DL_L_HKAA'].notnull()]


In [24]:
# rater avg surgical leg dataframe
df_avg_surg_left2 = df_surg_left2[['RECORD_ID', 'L_HKAA_avg_of_raters']].rename(columns = {'L_HKAA_avg_of_raters' : 'HKAA_surg'})
df_avg_surg_right2 = df_surg_right2[['RECORD_ID', 'R_HKAA_avg_of_raters']].rename(columns = {'R_HKAA_avg_of_raters' : 'HKAA_surg'})
df_avg_surg2 = pd.concat([df_avg_surg_left2, df_avg_surg_right2])
df_avg_surg2['rater_id'] = 1

In [25]:
# DL surgical leg dataframe
df_surg_left_dl2 = df_surg_left2[['RECORD_ID', 'DL_L_HKAA']].rename(columns = {'DL_L_HKAA' : 'HKAA_surg'})
df_surg_right_dl2 = df_surg_right2[['RECORD_ID', 'DL_R_HKAA']].rename(columns = {'DL_R_HKAA' : 'HKAA_surg'})
df_surg_dl2 = pd.concat([df_surg_left_dl2, df_surg_right_dl2])
df_surg_dl2['rater_id'] = 2

In [26]:
# rater avg non-surgical leg dataframe
df_avg_nsurg_left2 = df_surg_left2[df_surg_left2['DL_R_HKAA'].notnull()][['RECORD_ID', 'R_HKAA_avg_of_raters']].rename(columns = {'R_HKAA_avg_of_raters' : 'HKAA_nsurg'})
df_avg_nsurg_right2 = df_surg_right2[df_surg_right2['DL_L_HKAA'].notnull()][['RECORD_ID', 'L_HKAA_avg_of_raters']].rename(columns = {'L_HKAA_avg_of_raters' : 'HKAA_nsurg'})
df_avg_nsurg2 = pd.concat([df_avg_nsurg_left2, df_avg_nsurg_right2])
df_avg_nsurg2['rater_id'] = 1

In [27]:
# DL non-surgical leg dataframe
df_nsurg_left_dl2 = df_surg_left2[df_surg_left2['DL_R_HKAA'].notnull()][['RECORD_ID', 'DL_R_HKAA']].rename(columns = {'DL_R_HKAA' : 'HKAA_nsurg'})
df_nsurg_right_dl2 = df_surg_right2[df_surg_right2['DL_L_HKAA'].notnull()][['RECORD_ID', 'DL_L_HKAA']].rename(columns = {'DL_L_HKAA' : 'HKAA_nsurg'})
df_nsurg_dl2 = pd.concat([df_nsurg_left_dl2, df_nsurg_right_dl2])
df_nsurg_dl2['rater_id'] = 2

In [28]:
DL_surg_count2 = len(df_surg_dl2)
print(DL_surg_count2)

124


## IRR b/w DL and average human rater on the surgical vs non-surgical leg

### Surgical Leg

In [29]:
# human rater
# calculate avg
df_avg_surg2['HKAA_surg'].mean().round(3)

172.432

In [30]:
df_avg_surg2['HKAA_surg'].std().round(3)

4.851

In [31]:
# dl
# calculate avg
df_surg_dl2['HKAA_surg'].mean().round(3)

173.038

In [32]:
df_surg_dl2['HKAA_surg'].std().round(3)

4.524

In [33]:
# combine rater and dl
df_surg2 = pd.concat([df_avg_surg2, df_surg_dl2])
df_surg2

,RECORD_ID,HKAA_surg,rater_id
0,19,177.506,1
2,44,170.810,1
3,111,176.916,1
4,128,170.012,1
5,177,176.094,1
...,...,...,...
111,2687,179.183,2
112,2700,167.914,2
116,2732,179.377,2
118,2740,161.334,2


In [34]:
# IRR calculation
icc_surg2 = pg.intraclass_corr(data = df_surg2, targets = 'RECORD_ID', raters = 'rater_id', ratings = 'HKAA_surg').round(3)
icc_surg2.set_index('Type')

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.974,76.211,121,122,0.000,"[0.96, 0.98]"
ICC2,Single random raters,0.974,111.334,121,121,0.000,"[0.92, 0.99]"
ICC3,Single fixed raters,0.982,111.334,121,121,0.000,"[0.97, 0.99]"
ICC1k,Average raters absolute,0.987,76.211,121,122,0.000,"[0.98, 0.99]"
ICC2k,Average random raters,0.987,111.334,121,121,0.000,"[0.96, 0.99]"
ICC3k,Average fixed raters,0.991,111.334,121,121,0.000,"[0.99, 0.99]"


The interrater reliability is 0.991 for the surgical leg.

### Non-surgical Leg

In [35]:
# human rater
# calculate avg
df_avg_nsurg2['HKAA_nsurg'].mean().round(3)

175.211

In [36]:
df_avg_nsurg2['HKAA_nsurg'].std().round(3)

3.722

In [37]:
# dl
# calculate avg
df_nsurg_dl2['HKAA_nsurg'].mean().round(3)

175.531

In [38]:
df_nsurg_dl2['HKAA_nsurg'].std().round(3)

3.538

In [39]:
# combine rater and dl
df_nsurg2 = pd.concat([df_avg_nsurg2, df_nsurg_dl2])
df_nsurg2

,RECORD_ID,HKAA_nsurg,rater_id
0,19,178.882,1
2,44,170.961,1
3,111,179.792,1
4,128,176.671,1
5,177,174.843,1
...,...,...,...
111,2687,178.684,2
112,2700,172.768,2
116,2732,179.860,2
118,2740,163.669,2


In [40]:
# IRR calculation
icc_nsurg2 = pg.intraclass_corr(data = df_nsurg2, targets = 'RECORD_ID', raters = 'rater_id', ratings = 'HKAA_nsurg').round(3)
icc_nsurg2.set_index('Type')

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.974,74.866,120,121,0.000,"[0.96, 0.98]"
ICC2,Single random raters,0.974,86.487,120,120,0.000,"[0.95, 0.98]"
ICC3,Single fixed raters,0.977,86.487,120,120,0.000,"[0.97, 0.98]"
ICC1k,Average raters absolute,0.987,74.866,120,121,0.000,"[0.98, 0.99]"
ICC2k,Average random raters,0.987,86.487,120,120,0.000,"[0.98, 0.99]"
ICC3k,Average fixed raters,0.988,86.487,120,120,0.000,"[0.98, 0.99]"


The interrater reliability is 0.988 for the surgical leg.

# Task 9: DL model comparison

In [41]:
def calculate_z_test(count1, nobs1, count2, nobs2):
    count = [count1, count2]
    nobs = [nobs1, nobs2]
    z, p_value = sm.stats.proportions_ztest(count, nobs, alternative='two-sided')
    return z, p_value

# Operative Limb
count1_op = 124
nobs1_op = 124
count2_op = 156
nobs2_op = 163
z_op, p_value_op = calculate_z_test(count1_op, nobs1_op, count2_op, nobs2_op)
print(f"Operative Limb: z = {z_op:.4f}, p-value = {p_value_op:.4f}")

# Non-operative Limb
count1_non_op = 119
nobs1_non_op = 120
count2_non_op = 143
nobs2_non_op = 153
z_non_op, p_value_non_op = calculate_z_test(count1_non_op, nobs1_non_op, count2_non_op, nobs2_non_op)
print(f"Non-operative Limb: z = {z_non_op:.4f}, p-value = {p_value_non_op:.4f}")

Operative Limb: z = 2.3363, p-value = 0.0195
Non-operative Limb: z = 2.3782, p-value = 0.0174


In [42]:
# OP
# Neutral
# DL test vs DL difficult
z, p = calculate_z_test(30, 124, 27, 156)
print(f"z = {z:.3f}, p = {p:.3f}")

z = 1.421, p = 0.155


In [43]:
# OP
# Varus
# DL test vs DL difficult
z, p = calculate_z_test(77, 124, 110, 156)
print(f"z = {z:.3f}, p = {p:.3f}")

z = -1.485, p = 0.137


In [44]:
# OP
# Valgus
# DL test vs DL difficult
z, p = calculate_z_test(17, 124, 19, 156)
print(f"z = {z:.3f}, p = {p:.3f}")

z = 0.380, p = 0.704


In [45]:
# NOP
# Neutral
# DL test vs DL difficult
z, p = calculate_z_test(52, 119, 58, 143)
print(f"z = {z:.3f}, p = {p:.3f}")

z = 0.512, p = 0.608


In [46]:
# NOP
# Varus
# DL test vs DL difficult
z, p = calculate_z_test(60, 119, 75, 143)
print(f"z = {z:.3f}, p = {p:.3f}")

z = -0.327, p = 0.744


In [47]:
# NOP
# Valgus
# DL test vs DL difficult
z, p = calculate_z_test(7, 119, 10, 143)
print(f"z = {z:.3f}, p = {p:.3f}")

z = -0.363, p = 0.716
